In [44]:
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime
from pandas_datareader import data
from datetime import datetime, timedelta
import requests
from bs4 import BeautifulSoup
import re
import json


In [45]:
def get_12Moforecast(tick):
    """
    Takes in a tick sign and date variable and gets the historical dividend dates for that tick symbal AFTER the data input.
    Outputs: dataframe with these columns:
    'exOrEffDate', 'type', 'amount', 'declarationDate', 'recordDate','paymentDate', 'Tick'
    """
    
    #need headers otherwise they will think you are a bot 
    headers = { "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9", 
            "Upgrade-Insecure-Requests": "1",
            "User-Agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.183 Safari/537.36 RuxitSynthetic/1.0 v7992529445 t38550 ath9b965f92 altpub cvcv=2"
        }

    url = "https://money.cnn.com/quote/forecast/forecast.html?symb={}".format(tick)
    response = requests.get(url,headers=headers)
    
    return response

In [47]:
def parse_12Moforecast(tick, response):

    html_text = response.text
    
    columnNames = ['tick', 'Analysts', 'Recommendation', 'Median', 'High', 'Low', 'Growth', 'Last']
    
    soup = BeautifulSoup(html_text, "html.parser")
    
    # find all div elements that are inside a div element
    # and are proceeded by an h3 element
    selector = 'div > h3 ~ div'
    
    divs = soup.select(selector)
    
    # Extract data from the found elements
    data = [x.text.split(';')[-1].strip() for x in divs]
    
    df = pd.DataFrame(columns = columnNames)

    for x in data:
        
        if (x.find("12-month price forecasts for") > 0):
            matches = re.findall(r'[\d\.\d]+', x)
            df.loc[1, 'Median'] = matches[2]
            df.loc[1, 'High'] = matches[3]
            df.loc[1, 'Low'] = matches[4]
            df.loc[1, 'Growth'] = matches[5]
            df.loc[1, 'Last'] = matches[6]
            
        if (x.startswith("The current consensus among") == 1):
            match = re.findall(r'[\d\.\d]+', x)    
            df.loc[1, 'Analysts'] = match[1]
            if (x.find('is to Hold stock') > 0):
                df.loc[1, 'Recommendation'] = 'Hold'
            if (x.find('is to Buy stock') > 0):
                df.loc[1, 'Recommendation'] = 'Buy'
            if (x.find('is to Sell stock') > 0):
                df.loc[1, 'Recommendation'] = 'Sell'
    
    df.loc[1, 'tick'] = tick
           
    return df


In [48]:
def get_tickers():

    #need headers otherwise they will think you are a bot 
    headers = { "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9", 
            "Upgrade-Insecure-Requests": "1",
            "User-Agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.183 Safari/537.36 RuxitSynthetic/1.0 v7992529445 t38550 ath9b965f92 altpub cvcv=2"
        }

    url = "https://www.thestreet.com/util/ratings-screener.jsp?wt=json&rows=100000&indent=off&q=type:equity+AND+LetterGradeSort:[+80+TO+100+]+AND+Growth:[+0+TO+5+]++AND+TotalReturn:[+0+TO+5+]++AND+Efficiency:[+0+TO+5+]++AND+Pricevolatility:[+0+TO+5+]++AND+Solvency:[+0+TO+5+]++AND+Income:[+0+TO+5+]+&fl=ticker,issue_name,LetterGradeRating,CurrentRating,LetterGradeSort,type,Risk,recommendation,exchange&sort=ticker+asc"
    response = requests.get(url,headers=headers)

    json_data = json.loads(response.text)

    json_divs = json_data['response']['docs']
    df = pd.json_normalize(json_divs)
        
    return df

In [ ]:
dfs = []
ticks = []
ticks = get_tickers()

for stock in ticks['ticker']:
    response = get_12Moforecast(stock)
    df = parse_12Moforecast(stock, response)
    
    if (df.loc[1, 'Recommendation'] == ('Buy')): 
        try:
            float_growth = float(df.loc[1, 'Growth'])
            # only interested if median growth > 30%
            if(float_growth >= 30.0):

                print("Ticker:{} Growth:{}%".format(stock, float_growth))
print(df)
                dfs.append(df)   
        
        except ValueError:
            print("Oops!  Value for {} ticker can't be converted to float - {}".format(stock, df.loc[1, 'Growth']))
            print(df)
        


Ticker:A Growth:1.58%
Ticker:AAP Growth:4.71%
Ticker:AAPL Growth:4.81%
Ticker:AB Growth:5.85%
Ticker:ABBV Growth:7.97%
Ticker:ABG Growth:17.5%
Ticker:ABT Growth:12.11%
   tick Analysts Recommendation Median  High     Low Growth   Last
1  ACCO        .            Buy   12.0  14.0  10.00.  33.19  9.01.
Ticker:ACCO Growth:33.19%
Ticker:ACGL Growth:9.95%
Ticker:ACMR Growth:25.14%
Ticker:ACN Growth:9.51%
Ticker:ADBE Growth:20.96%


In [ ]:
result = pd.concat(dfs, ignore_index=True)
result.to_csv('12MoForecast.csv',index=False)
result